In [ ]:
# !pip install yfinance --upgrade

import yfinance as yf

# Pick a stock
ticker = yf.Ticker("AAPL")

# Get upcoming and past earnings dates
earnings_dates = ticker.get_earnings_dates(limit=10)

future_earnings = earnings_dates[earnings_dates["Reported EPS"].isna()]
next_earnings = future_earnings.index[0]

print("Next earnings date:", next_earnings.date())

print(earnings_dates)


Next earnings date: 2025-02-27
                           EPS Estimate  Reported EPS  Surprise(%) Event Type
Earnings Date                                                                
2025-05-01 16:30:00-04:00          1.63          1.65         1.41   Earnings
2025-02-27 12:00:00-05:00           NaN           NaN          NaN    Meeting
2025-01-30 16:31:00-05:00          2.35          2.40         2.15   Earnings
2024-10-31 16:31:00-04:00          1.60          1.64         2.35   Earnings
2024-08-01 16:30:00-04:00          1.35          1.40         3.99   Earnings
2024-05-02 16:31:00-04:00          1.50          1.53         1.97   Earnings
2024-02-01 16:00:00-05:00          2.10          2.18         3.90   Earnings
2023-11-02 16:30:00-04:00          1.39          1.46         4.92   Earnings
2023-08-03 16:30:00-04:00          1.19          1.26         5.49   Earnings
2023-05-04 16:30:00-04:00          1.43          1.52         6.03   Earnings


In [4]:
!pip install yahoo_earnings_calendar

import datetime
from yahoo_earnings_calendar import YahooEarningsCalendar

yec = YahooEarningsCalendar()
future_ts = yec.get_next_earnings_date("AAPL")  # returns timestamp
future_date = datetime.datetime.fromtimestamp(future_ts)
print("Next earnings:", future_date)

Exception: Invalid Symbol or Unavailable Earnings Date